**Cleaning Continues - Hillary Clinton Tweets**

    1. Note Clinton tweets are only from 2015 to 2016 for the election period, this will be explained more later on.  
    2. Import the needed libraries
    3. Import CSV of Hillary Clinton tweets and create a dataframe df_clint
    4. Initial Analysis of the df

In [1]:
import pandas as pd
import numpy as np
import sidetable #we might not use it, but just in case
import datetime

In [2]:
df_clint = pd.read_csv('../data/tweets.csv')
df_clint.head(2)

,id,handle,text,is_retweet,original_author,time,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,is_quote_status,...,place_type,place_country_code,place_country,place_contained_within,place_attributes,place_bounding_box,source_url,truncated,entities,extended_entities
0,780925634159796224,HillaryClinton,The question in this election: Who can put the...,False,NaN,2016-09-28T00:22:34,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,https://studio.twitter.com,False,{'media': [{'display_url': 'pic.twitter.com/Xr...,{'media': [{'display_url': 'pic.twitter.com/Xr...
1,780916180899037184,HillaryClinton,"Last night, Donald Trump said not paying taxes...",True,timkaine,2016-09-27T23:45:00,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,http://twitter.com,False,{'media': [{'display_url': 'pic.twitter.com/t0...,{'media': [{'display_url': 'pic.twitter.com/t0...


In [3]:
df_clint.shape

(6444, 28)

In [4]:
#let´s see how many nulls
(df_clint.isnull().sum() / df_clint.shape[0]) * 100

id                          0.000000
handle                      0.000000
text                        0.000000
is_retweet                  0.000000
original_author            88.795779
time                        0.000000
in_reply_to_screen_name    96.772191
in_reply_to_status_id      96.865301
in_reply_to_user_id        96.772191
is_quote_status             0.000000
lang                        0.000000
retweet_count               0.000000
favorite_count              0.000000
longitude                  99.813780
latitude                   99.813780
place_id                   96.834264
place_full_name            96.834264
place_name                 96.834264
place_type                 96.834264
place_country_code         96.834264
place_country              96.834264
place_contained_within     96.834264
place_attributes           96.834264
place_bounding_box         96.834264
source_url                  0.000000
truncated                   0.000000
entities                    0.000000
e

In [5]:
df_clint.columns

#A lot of columns I dont really care

Index(['id', 'handle', 'text', 'is_retweet', 'original_author', 'time',
       'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_user_id', 'is_quote_status', 'lang', 'retweet_count',
       'favorite_count', 'longitude', 'latitude', 'place_id',
       'place_full_name', 'place_name', 'place_type', 'place_country_code',
       'place_country', 'place_contained_within', 'place_attributes',
       'place_bounding_box', 'source_url', 'truncated', 'entities',
       'extended_entities'],
      dtype='object')

In [6]:
#let´s delete

df_clint.drop(['in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_user_id', 'is_quote_status','longitude', 'latitude', 'place_id',
       'place_full_name', 'place_name', 'place_type', 'place_country_code',
       'place_country', 'place_contained_within', 'place_attributes','is_retweet',
        'original_author','place_bounding_box', 'source_url', 'truncated', 'entities',
       'extended_entities','lang'], axis=1, inplace = True)

In [7]:
df_clint.head(2)

,id,handle,text,time,retweet_count,favorite_count
0,780925634159796224,HillaryClinton,The question in this election: Who can put the...,2016-09-28T00:22:34,218,651
1,780916180899037184,HillaryClinton,"Last night, Donald Trump said not paying taxes...",2016-09-27T23:45:00,2445,5308


**Note_2:** columns are homogeneous :) , the 'id' column can be checked as unique ID to be replaced:

    1.First let me check if there are any duplicates to see if I should make it as the id column
    2.Set id column for 'id'

In [8]:
#me gustaría ver si el id tiene duplicado, y después si no tiene, lo pongo como index

df_clint.duplicated('id').sum()

0

In [9]:
#buenisimo, ahora a setearlo como index

df_clint.set_index(['id'], inplace = True)
df_clint.head(2)

,handle,text,time,retweet_count,favorite_count
id,,,,,
780925634159796224,HillaryClinton,The question in this election: Who can put the...,2016-09-28T00:22:34,218,651
780916180899037184,HillaryClinton,"Last night, Donald Trump said not paying taxes...",2016-09-27T23:45:00,2445,5308


In [10]:
#oh this is looking nice, but oh wait, let´s check the types of the data itself.

df_clint.dtypes

handle            object
text              object
time              object
retweet_count      int64
favorite_count     int64
dtype: object

**Note_3**: the dates need to be changed, I will:

        1. Declare a variable 'x' which will be the pattern of the date and time I want to split. 
        2. Create 2 functions 'limpiar_fecha' and 'limpiar_hora' to clean both dates and hours. 
        3. Apply the formula using an .apply() and create respective columns [date] and [hora]
        4. Create new columns using datetime():
            a. Year
            b. Month
            c. Day
            d. Hour
            e. Minute

In [11]:
#1. Declare and set variable
x = '2016-09-28T00:22:34'
x.split("T")

['2016-09-28', '00:22:34']

In [12]:
#build functions 
def limpiar_fecha(x): 
    fecha_string1 = x.split("T")[0]
    fecha_date = datetime.datetime.strptime(fecha_string1, '%Y-%m-%d')
    return fecha_date

def limpiar_hora(x): 
    fecha_string1 = x.split("T")[1]
    fecha_hora = datetime.datetime.strptime(fecha_string1, '%H:%M:%S')
    return fecha_hora

In [13]:
#apply functions
df_clint["date"] = df_clint["time"].apply(limpiar_fecha)
df_clint["hora"] = df_clint["time"].apply(limpiar_hora)

In [14]:
df_clint.head(2)

,handle,text,time,retweet_count,favorite_count,date,hora
id,,,,,,,
780925634159796224,HillaryClinton,The question in this election: Who can put the...,2016-09-28T00:22:34,218,651,2016-09-28,1900-01-01 00:22:34
780916180899037184,HillaryClinton,"Last night, Donald Trump said not paying taxes...",2016-09-27T23:45:00,2445,5308,2016-09-27,1900-01-01 23:45:00


In [15]:
#confirm
df_clint.dtypes

handle                    object
text                      object
time                      object
retweet_count              int64
favorite_count             int64
date              datetime64[ns]
hora              datetime64[ns]
dtype: object

In [16]:
#now to get new columns per date

df_clint['year'] = df_clint['date'].dt.year
df_clint['month'] = df_clint['date'].dt.month
df_clint['day'] = df_clint['date'].dt.day
df_clint['hour'] = df_clint['hora'].dt.hour
df_clint['minute'] = df_clint['hora'].dt.minute

In [17]:
df_clint.head(2)

,handle,text,time,retweet_count,favorite_count,date,hora,year,month,day,hour,minute
id,,,,,,,,,,,,
780925634159796224,HillaryClinton,The question in this election: Who can put the...,2016-09-28T00:22:34,218,651,2016-09-28,1900-01-01 00:22:34,2016,9,28,0,22
780916180899037184,HillaryClinton,"Last night, Donald Trump said not paying taxes...",2016-09-27T23:45:00,2445,5308,2016-09-27,1900-01-01 23:45:00,2016,9,27,23,45


**Note_4:** Here I like how the other columns came out, but I have to clean up the dataframe:

        1. Drop columns date, time and hora
        2. Double check I only have Clinton tweets using stb.freq on the 'handles'
        3. If there are Donald Trump tweets, I will then:
            a. Drop the tweets where the handle is 'realDonaldTrump'
            b. Confirm they are dropped
        4. Save new file

In [18]:
df_clint.drop(['date','time','hora'], axis = 1, inplace = True)

In [19]:
df_clint.head(2)

,handle,text,retweet_count,favorite_count,year,month,day,hour,minute
id,,,,,,,,,
780925634159796224,HillaryClinton,The question in this election: Who can put the...,218,651,2016,9,28,0,22
780916180899037184,HillaryClinton,"Last night, Donald Trump said not paying taxes...",2445,5308,2016,9,27,23,45


In [20]:
#now I want to check the handles, because I only care about Hillary´s tweets

df_clint.stb.freq(['handle'])

,handle,count,percent,cumulative_count,cumulative_percent
0,HillaryClinton,3226,50.062073,3226,50.062073
1,realDonaldTrump,3218,49.937927,6444,100.000000


In [21]:
#dropping donald trump tweets

df_clint.drop(df_clint[df_clint.handle == 'realDonaldTrump'].index, inplace = True)

In [22]:
#confirmation

df_clint.stb.freq(['handle'])

,handle,count,percent,cumulative_count,cumulative_percent
0,HillaryClinton,3226,100.0,3226,100.0


In [23]:
#time to save it

df_clint.to_csv('../data/clintonclean.csv')